# MNIST Handwritten Digit Recognition
Depends on MNIST in CSV format: http://yann.lecun.com/exdb/mnist/
    


In [2]:
#imports
import numpy as np 
import pandas as pd 
import matplotlib.pylab as plt

import keras
from keras.datasets import mnist
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.models import Sequential
from keras.utils import to_categorical


Using TensorFlow backend.


In [3]:
#!ls
#!pwd
#!ls ..
#!ls ../input
#example_file = "../input/sample_submission.csv"
#df = pd.read_csv(example_file)
#print(df.head())

In [4]:
#constants
#base_path = '../input/'
base_path = ''
train_file = base_path + 'train.csv'
test_file = base_path + 'test.csv'

num_classes = 10
dev_set_fraction = 0.05

In [5]:
#explore
df = pd.read_csv(train_file)
print(df.head())
print("shape: " + str(df.shape))
df.describe()

dft = pd.read_csv(test_file)
print(dft.head())
print("shape: " + str(dft.shape))
dft.describe()


   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8    ...     pixel774  pixel775  pixel776  pixel777  pixel778  \
0       0    ...            0         0         0         0         0   
1       0    ...            0         0         0         0         0   
2       0    ...            0         0         0         0         0   
3       0    ...            0         0         0         0         0   
4       0    ...            0         0         0         0         0   

   pixel779  pixel780  pixel781  pixel782  pixel783  
0         0         0         0         0         

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,28000.0,28000.0,28000.0,28000.0,28000.0,28000.0,28000.0,28000.0,28000.0,28000.0,...,28000.000000,28000.000000,28000.000000,28000.000000,28000.000000,28000.0,28000.0,28000.0,28000.0,28000.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.164607,0.073214,0.028036,0.011250,0.006536,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.473293,3.616811,1.813602,1.205211,0.807475,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,253.000000,254.000000,193.000000,187.000000,119.000000,0.0,0.0,0.0,0.0,0.0


In [6]:
# extract
df = pd.read_csv(train_file)

m = df.shape[0]
pixelnum = df.shape[1]-1

# randomize so we can split again to get a dev set
df = df.sample(frac=1).reset_index(drop=True)

x_train = df.iloc[:, 1:pixelnum+1].values
y_train = to_categorical(df['label'].values, num_classes=10)

split = int(m - m*dev_set_fraction)
x = x_train[:split, :]
y = y_train[:split, :]
x_dev = x_train[split:, :]
y_dev = y_train[split:, :]

print("x: {}, y: {}".format(x.shape, y.shape))

print("x_dev: {}, y_dev: {}".format(x_dev.shape, y_dev.shape))

dftest = pd.read_csv(test_file)
x_test = dftest.values
print("x_test: {}".format(x_test.shape))

x: (39900, 784), y: (39900, 10)
x_dev: (2100, 784), y_dev: (2100, 10)
x_test: (28000, 784)


In [7]:
# model structure
model = Sequential()

model.add(Dense(784, activation='tanh', name="d1",  input_dim=pixelnum))
model.add(Dense(392, activation='tanh', name="d2"))
model.add(Dense(num_classes, activation='softmax', name="d3"))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [8]:
#fit
model.fit(x, y,
          batch_size=128,
          epochs=3,
          verbose=1
          ,validation_data=(x_dev, y_dev)
         )
         
score = model.evaluate(x_dev, y_dev, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 39900 samples, validate on 2100 samples
Epoch 1/3
39900/39900 [==============================] - 8s 213us/step - loss: 0.3733 - acc: 0.8830 - val_loss: 0.2941 - val_acc: 0.9138
Epoch 2/3
39900/39900 [==============================] - 9s 225us/step - loss: 0.2861 - acc: 0.9105 - val_loss: 0.2803 - val_acc: 0.9086
Epoch 3/3
39900/39900 [==============================] - 9s 220us/step - loss: 0.2527 - acc: 0.9207 - val_loss: 0.2501 - val_acc: 0.9257
Test loss: 0.25007747996421087
Test accuracy: 0.9257142856007531


In [9]:
y_test = model.predict(x_test)
y_test = np.argmax(y_test, axis=1).astype(int)
y_test = pd.DataFrame({"ImageId": np.arange(y_test.shape[0])+1,"Label": y_test})
print(y_test.head())
y_test.to_csv("kaggle_submission.csv", sep=",", index=False)

   ImageId  Label
0        1      2
1        2      0
2        3      9
3        4      2
4        5      3
